In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

/Users/lukethomas/Documents/searchyenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 21.4kB/s]
Downloading: 100%|██████████| 829/829 [00:00<00:00, 609kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 852kB/s] 
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.65kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 97.9kB/s]
Downloading: 100%|██████████| 433M/433M [00:06<00:00, 62.7MB/s] 


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]
